In [1]:
# tqdm 설치 (진행률 바 표시를 위한 라이브러리)
!pip install tqdm

In [2]:
!curl -o aihubshell https://api.aihub.or.kr/api/aihubshell.do

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  6336  100  6336    0     0   5276      0  0:00:01  0:00:01 --:--:--  5280


In [3]:
!chmod +x aihubshell


In [4]:
import os
os.environ['AIHUB_API_KEY'] = 'C273D7AC-ECFB-421E-9E0D-93CBF18EAE43'

In [5]:
!./aihubshell -mode l -datasetkey 71557

aihubshell version 25.02.12 v0.5
Fetching file tree structure...
The contents are encoded in UTF-8 including Korean characters. 
If the following contents are not output normally, 
Please modify the character information of the OS. 
=================공지사항=================== 

    └─138.뉴스 대본 및 앵커 음성 데이터
        └─01-1.정식개방데이터
            ├─Training
            │  ├─01.원천데이터
            │  │  ├─TS.z01 | 100 GB | 470564
            │  │  ├─TS.z02 | 100 GB | 470565
            │  │  └─TS.zip | 2 GB | 470566
            │  └─02.라벨링데이터
            │      └─TL.zip | 244 MB | 470567
            └─Validation
                ├─01.원천데이터
                │  └─VS.zip | 25 GB | 470568
                └─02.라벨링데이터
                    └─VL.zip | 28 MB | 470569


In [6]:
!./aihubshell -mode d -datasetkey 71557 -filekey 470567,470569  -aihubapikey 'C273D7AC-ECFB-421E-9E0D-93CBF18EAE43'

aihubshell version 25.02.12 v0.5
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    82  100    82    0     0    101      0 --:--:-- --:--:-- --:--:--   101
Download failed with HTTP status 502.
Error msg:
AI 허브는 해외에서의 데이터 다운로드를 제한하고 있습니다.


In [7]:
!curl -s https://ipinfo.io

{
  "ip": "34.86.175.23",
  "hostname": "23.175.86.34.bc.googleusercontent.com",
  "city": "Washington",
  "region": "District of Columbia",
  "country": "US",
  "loc": "38.8951,-77.0364",
  "org": "AS396982 Google LLC",
  "postal": "20004",
  "timezone": "America/New_York",
  "readme": "https://ipinfo.io/missingauth"
}

- Google Colab은 사용자의 물리적 위치와 관계없이 글로벌 서버를 사용합니다.
- 한국 내에서 Colab에 접속하더라도, Colab 런타임은 해외 서버에서 실행될 수 있습니다.
- AIHub는 Colab 서버의 IP를 해외 IP로 인식하여 다운로드를 차단하고 있습니다.


# colab의 무료버전은 서버위치를 고정할수 없으므로 다운로드가 되었다는 가정하에 진행하였습니다

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
import os

# 폴더 생성
os.makedirs('/content/TL', exist_ok=True)
os.makedirs('/content/VL', exist_ok=True)

print("폴더가 성공적으로 생성되었습니다.")


폴더가 성공적으로 생성되었습니다.


In [17]:
import zipfile
import os
from tqdm import tqdm  # 진행률 바 표시를 위한 라이브러리

# 압축 파일 경로 및 해제할 폴더 경로
zip_file_path = '/content/drive/MyDrive/138.뉴스 대본 및 앵커 음성 데이터/01-1.정식개방데이터/Training/02.라벨링데이터/TL.zip'
extract_to_path = '/content/TL'

# 압축 해제 진행률 표시
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    # 전체 파일 개수 확인
    total_files = len(zip_ref.infolist())

    # tqdm을 사용하여 진행률 표시
    with tqdm(total=total_files, desc='Extracting', unit='file') as pbar:
        for file in zip_ref.infolist():
            zip_ref.extract(file, extract_to_path)
            pbar.update(1)  # 진행률 업데이트

print("TL압축 파일이 성공적으로 해제되었습니다.")

zip_file_path = '/content/drive/MyDrive/138.뉴스 대본 및 앵커 음성 데이터/01-1.정식개방데이터/Validation/02.라벨링데이터/VL.zip'
extract_to_path = '/content/VL'

# 압축 해제 진행률 표시
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    # 전체 파일 개수 확인
    total_files = len(zip_ref.infolist())

    # tqdm을 사용하여 진행률 표시
    with tqdm(total=total_files, desc='Extracting', unit='file') as pbar:
        for file in zip_ref.infolist():
            zip_ref.extract(file, extract_to_path)
            pbar.update(1)  # 진행률 업데이트

print("VL압축 파일이 성공적으로 해제되었습니다.")

Extracting: 100%|██████████| 343974/343974 [00:59<00:00, 5767.22file/s]


TL압축 파일이 성공적으로 해제되었습니다.


Extracting: 100%|██████████| 40321/40321 [00:07<00:00, 5511.28file/s]

VL압축 파일이 성공적으로 해제되었습니다.


# 라벨 데이터(json)를 모두 읽어서 음성을 전사한 부분만 추출해서 하나의 text_only.txt 에 저장하는 코드 작성

## TL

In [18]:
import json


# JSON 파일들이 저장된 최상위 디렉토리 경로
base_dir = '/content/TL'
output_file = '/content/TL/text_only.txt'

# text_only.txt 초기화
with open(output_file, 'w') as f:
    f.write('')  # 내용 초기화


# 모든 하위 디렉토리를 탐색하며 JSON 파일 개수 세기
json_files = []
for root, dirs, files in os.walk(base_dir):
    for file_name in files:
        if file_name.endswith('.json'):
            json_files.append(os.path.join(root, file_name))

# 총 파일 개수
total_files = len(json_files)

# tqdm 진행률 바 설정
with tqdm(total=total_files, desc='Processing JSON files', unit='file') as pbar:
    # JSON 파일 읽기 및 전사텍스트 추출
    for json_file_path in json_files:
        with open(json_file_path, 'r', encoding='utf-8') as json_file:
            data = json.load(json_file)

            # 파일 이름 추출 (확장자 제외)
            file_name = os.path.basename(json_file_path).replace('.json', '')

            # text 필드 가져오기
            transcription = data.get('script', {}).get('text', None)

            # 둘 다 존재할 때만 기록
            if file_name and transcription:
                line = f"{file_name}:{transcription}\n"

                # text_only.txt에 기록
                with open(output_file, 'a', encoding='utf-8') as out_file:
                    out_file.write(line)

        # tqdm 진행률 업데이트
        pbar.update(1)

print("\n 모든 JSON 파일에서 전사텍스트를 추출하여 text_only.txt에 저장했습니다.")


Processing JSON files: 100%|██████████| 306734/306734 [00:21<00:00, 13975.39file/s]


 모든 JSON 파일에서 전사텍스트를 추출하여 text_only.txt에 저장했습니다.


In [19]:
import re

# 입력 파일 및 출력 파일 경로
input_file = '/content/TL/text_only.txt'
output_file = '/content/TL_text_preprocessed.txt'

# text_preprocessed.txt 초기화
with open(output_file, 'w') as f:
    f.write('')  # 내용 초기화

# 전처리 함수
def preprocess_text(text):
    # (숫자)/(발화) 패턴을 찾아 발화만 남김
    processed_text = re.sub(r'\((\d+)\)/\(([^)]+)\)', r'\2', text)
    return processed_text

# 숫자만 남기는 전처리 함수
def preprocess_text_number_only(text):
    # (숫자)/(발화) 패턴을 찾아 숫자만 남김
    processed_text = re.sub(r'\((\d+)\)/\([^)]+\)', r'\1', text)
    return processed_text

# 입력 파일 읽기 및 전처리
with open(input_file, 'r', encoding='utf-8') as f:
    lines = f.readlines()
    for line in lines:
        # audio_name과 원본 텍스트 분리
        if ':' in line:
            audio_name, original_text = line.split(':', 1)
            original_text = original_text.strip()

            # 텍스트 전처리
            preprocessed_text = preprocess_text(original_text)
            preprocessed_text_number = preprocess_text_number_only(original_text)

            # 결과 작성
            result_line = f"{audio_name}:{preprocessed_text_number} ${preprocessed_text}\n"

            # text_preprocessed.txt에 기록
            with open(output_file, 'a', encoding='utf-8') as out_file:
                out_file.write(result_line)

print("텍스트 전처리가 완료되었습니다. 결과는 text_preprocessed.txt에 저장되었습니다.")

텍스트 전처리가 완료되었습니다. 결과는 text_preprocessed.txt에 저장되었습니다.


In [24]:
# text_only.txt 첫 5줄 확인
print("=== text_only.txt 첫 5줄 ===")
!head -n 5 /content/TL/text_only.txt

# 줄바꿈 추가
print("\n")

# text_preprocessed.txt 첫 5줄 확인
print("=== text_preprocessed.txt 첫 5줄 ===")
!head -n 5 /content/TL_text_preprocessed.txt


=== text_only.txt 첫 5줄 ===
SPK051YTNPO680M002:세 후보는 부동산 안정화 대책과 행정수도 이전 등의 현안에서는 크게 다르지 않은 목소리를 냈지만, 내년 재보궐 선거에 서울과 부산시장 후보 공천을 해야할지를 두고는 시각차를 보였습니다.
SPK051YTNPO680M008:세 후보는 내일 부산,울산,경남, 다음 달 (6)/(육) 일 전북, (7)/(칠) 일 광주,전남, (13)/(십 삼) 일 충청 지역에서 권역별 (TV)/(티 브이) 토론을 이어갑니다.
SPK051YTNPO680M004:하지만 김부겸 후보는 이후 치러질 대선을 고려해서라도 후보를 공천하고 지도부가 비판을 막아주는 역할을 해야 한다며 반대 의견을 내놨습니다.
SPK051YTNPO680M003:이낙연 후보는 시급한 현안이 많은 만큼 지금 공천 여부를 논의하기보다는 연말쯤 결정하는 것이 좋다고 밝혔고, 박주민 후보는 차기 지도부가 결정할 문제지만, 국민 설득 시간이 필요할 수 있는 만큼 연말보다는 빨리 결정할 필요가 있다고 답했습니다.
SPK051YTNPO680M007:또 이와 관련해 박 후보가 대법원은 대구로 이전하고 헌법재판소는 광주로 이전하는 방안이 필요하다고 강조했고, 이 후보 역시 헌재와 대법원 이전도 검토할 가치가 있다고 생각한다고 덧붙였습니다.


=== text_preprocessed.txt 첫 5줄 ===
SPK051YTNPO680M002:세 후보는 부동산 안정화 대책과 행정수도 이전 등의 현안에서는 크게 다르지 않은 목소리를 냈지만, 내년 재보궐 선거에 서울과 부산시장 후보 공천을 해야할지를 두고는 시각차를 보였습니다. $세 후보는 부동산 안정화 대책과 행정수도 이전 등의 현안에서는 크게 다르지 않은 목소리를 냈지만, 내년 재보궐 선거에 서울과 부산시장 후보 공천을 해야할지를 두고는 시각차를 보였습니다.
SPK051YTNPO680M008:세 후보는 내일 부산,울산,경남, 다음 달 6 일 전북, 7 일 광주,전남, 13 일 충청 지역에서 권역별 (TV)/

In [25]:
# ()/() 패턴이 포함된 줄만 확인
print("=== text_only.txt ()/() 패턴이 포함된 줄중에서 첫 5줄 ===")
!grep -E "\(.*\)/\(.*\)" /content/TL/text_only.txt | head -n 5

# 줄바꿈 추가
print("\n")


print("=== text_preprocessed.txt ()/() 패턴이 포함된 줄중에서 첫 5줄 ===")
!grep -E "\(.*\)/\(.*\)" /content/TL_text_preprocessed.txt | head -n 5


=== text_only.txt ()/() 패턴이 포함된 줄중에서 첫 5줄 ===
SPK051YTNPO680M008:세 후보는 내일 부산,울산,경남, 다음 달 (6)/(육) 일 전북, (7)/(칠) 일 광주,전남, (13)/(십 삼) 일 충청 지역에서 권역별 (TV)/(티 브이) 토론을 이어갑니다.
SPK051YTNPO680M001:더불어민주당 당권에 도전한 이낙연, 김부겸, 박주민 후보가 대구광역시에서 첫 (TV)/(티 브이) 토론회를 열고 지지를 호소했습니다.
SPK051YTNCU674M006:다만 코로나 (19)/(일 구) 확산이 지속되고 있는 만큼 안전한 관람을 위해 전시장 내 동시관람 인원 (100)/(백) 인 제한은 유지됩니다.
SPK051YTNCU674M003:무료로 관람하는 `이건희컬렉션 특별전` 은 지난해 (7)/(칠) 월 (21)/(이십 일) 일 개막했고, 당초 지난달 (13)/(십 삼) 일 종료할 예정이었으나, 관람객이 꾸준히 이어지면서 한 달 연장된 바 있습니다.
SPK051YTNCU674M005:회차별 (100)/(백) 명씩 사전예약을 통해 운영하던 기존 방식에서, (14)/(십 사) 일부터는 온라인 사전예약 없이 (1)/(일) 인 (1)/(일) 매에 한해 현장 발권으로 관람할 수 있습니다.


=== text_preprocessed.txt ()/() 패턴이 포함된 줄중에서 첫 5줄 ===
SPK051YTNPO680M008:세 후보는 내일 부산,울산,경남, 다음 달 6 일 전북, 7 일 광주,전남, 13 일 충청 지역에서 권역별 (TV)/(티 브이) 토론을 이어갑니다. $세 후보는 내일 부산,울산,경남, 다음 달 육 일 전북, 칠 일 광주,전남, 십 삼 일 충청 지역에서 권역별 (TV)/(티 브이) 토론을 이어갑니다.
SPK051YTNPO680M001:더불어민주당 당권에 도전한 이낙연, 김부겸, 박주민 후보가 대구광역시에서 첫 (TV)/(티 브이) 토론회를 열고 지지를 호소했습니다. $더불어민주당 당권에 도전한 이낙연, 김부겸, 박주민 

## VL

In [22]:
import json


# JSON 파일들이 저장된 최상위 디렉토리 경로
base_dir = '/content/VL'
output_file = '/content/VL/text_only.txt'

# text_only.txt 초기화
with open(output_file, 'w') as f:
    f.write('')  # 내용 초기화


# 모든 하위 디렉토리를 탐색하며 JSON 파일 개수 세기
json_files = []
for root, dirs, files in os.walk(base_dir):
    for file_name in files:
        if file_name.endswith('.json'):
            json_files.append(os.path.join(root, file_name))

# 총 파일 개수
total_files = len(json_files)

# tqdm 진행률 바 설정
with tqdm(total=total_files, desc='Processing JSON files', unit='file') as pbar:
    # JSON 파일 읽기 및 전사텍스트 추출
    for json_file_path in json_files:
        with open(json_file_path, 'r', encoding='utf-8') as json_file:
            data = json.load(json_file)

            # 파일 이름 추출 (확장자 제외)
            file_name = os.path.basename(json_file_path).replace('.json', '')

            # text 필드 가져오기
            transcription = data.get('script', {}).get('text', None)

            # 둘 다 존재할 때만 기록
            if file_name and transcription:
                line = f"{file_name}:{transcription}\n"

                # text_only.txt에 기록
                with open(output_file, 'a', encoding='utf-8') as out_file:
                    out_file.write(line)

        # tqdm 진행률 업데이트
        pbar.update(1)

print("\n 모든 JSON 파일에서 전사텍스트를 추출하여 text_only.txt에 저장했습니다.")


Processing JSON files: 100%|██████████| 35126/35126 [00:03<00:00, 11298.45file/s]


 모든 JSON 파일에서 전사텍스트를 추출하여 text_only.txt에 저장했습니다.


In [23]:
import re

# 입력 파일 및 출력 파일 경로
input_file = '/content/VL/text_only.txt'
output_file = '/content/VL_text_preprocessed.txt'

# text_preprocessed.txt 초기화
with open(output_file, 'w') as f:
    f.write('')  # 내용 초기화

# 전처리 함수
def preprocess_text(text):
    # (숫자)/(발화) 패턴을 찾아 발화만 남김
    processed_text = re.sub(r'\((\d+)\)/\(([^)]+)\)', r'\2', text)
    return processed_text

# 입력 파일 읽기 및 전처리
with open(input_file, 'r', encoding='utf-8') as f:
    lines = f.readlines()
    for line in lines:
        # audio_name과 원본 텍스트 분리
        if ':' in line:
            audio_name, original_text = line.split(':', 1)
            original_text = original_text.strip()

            # 텍스트 전처리
            preprocessed_text = preprocess_text(original_text)

            # 결과 작성
            result_line = f"{audio_name}:{original_text} ${preprocessed_text}\n"

            # text_preprocessed.txt에 기록
            with open(output_file, 'a', encoding='utf-8') as out_file:
                out_file.write(result_line)

print("텍스트 전처리가 완료되었습니다. 결과는 text_preprocessed.txt에 저장되었습니다.")

텍스트 전처리가 완료되었습니다. 결과는 text_preprocessed.txt에 저장되었습니다.


In [20]:
# text_only.txt 첫 5줄 확인
print("=== text_only.txt 첫 5줄 ===")
!head -n 5 /content/VL/text_only.txt

# 줄바꿈 추가
print("\n")

# text_preprocessed.txt 첫 5줄 확인
print("=== text_preprocessed.txt 첫 5줄 ===")
!head -n 5 /content/VL/text_preprocessed.txt


=== text_only.txt 첫 5줄 ===
SPK051YTNPO786M001:앤드루 김 전 미국 중앙정보국, (CIA)/(씨 아이 에이) 코리아미션센터장은 북한이 미국과 대화 의지가 있고 행동 대 행동 접근법 등 미국의 호의적인 입장 표명을 기다릴지 모른다고 밝혔습니다.
SPK051YTNPO786M002:또 한국의 대선을 언급하며 북한이 향후 몇 달간 한국 문제에 집중할 것이라고 내다봤습니다.
SPK051YTNPO786M004:이어 북한의 이런 태도가 앞으로 미국과 일정한 협상을 계속하려는 희망을 갖고 있다는 의미라면서 미국이 북한과 소통할 호의적 방법을 보여주길 희망하는 것이라고 분석했습니다.
SPK051YTNPO786M003:김 전 센터장은 현지 시각 (5)/(오) 일 미국 워싱턴타임스재단이 주최한 화상 세미나에 참석해 '북한이 미국을 향해 완전한 도발의 사이클 대신 여전히 로키를 보이고 있다' 며 이 같이 말했습니다.
SPK051YTNPO786M005:김 전 센터장은 또 북한이 지금 당장은 한국의 국내 정치에 집중하고 있다면서 북한은 한국 정치에 영향력을 갖고 있다고 생각하고 있기 때문이라고 분석한 뒤 최근 남북 통신연락선 복원을 사례로 꼽았습니다.


=== text_preprocessed.txt 첫 5줄 ===
SPK051YTNPO786M001:앤드루 김 전 미국 중앙정보국, (CIA)/(씨 아이 에이) 코리아미션센터장은 북한이 미국과 대화 의지가 있고 행동 대 행동 접근법 등 미국의 호의적인 입장 표명을 기다릴지 모른다고 밝혔습니다. $앤드루 김 전 미국 중앙정보국, (CIA)/(씨 아이 에이) 코리아미션센터장은 북한이 미국과 대화 의지가 있고 행동 대 행동 접근법 등 미국의 호의적인 입장 표명을 기다릴지 모른다고 밝혔습니다.
SPK051YTNPO786M002:또 한국의 대선을 언급하며 북한이 향후 몇 달간 한국 문제에 집중할 것이라고 내다봤습니다. $또 한국의 대선을 언급하며 북한이 향후 몇 달간 한국 문제에 집중할 것이라고 내다봤습니다.
S

In [26]:
# ()/() 패턴이 포함된 줄만 확인
print("=== text_only.txt ()/() 패턴이 포함된 줄중에서 첫 5줄 ===")
!grep -E "\(.*\)/\(.*\)" /content/VL/text_only.txt | head -n 5

# 줄바꿈 추가
print("\n")


print("=== text_preprocessed.txt ()/() 패턴이 포함된 줄중에서 첫 5줄 ===")
!grep -E "\(.*\)/\(.*\)" /content/VL_text_preprocessed.txt | head -n 5


=== text_only.txt ()/() 패턴이 포함된 줄중에서 첫 5줄 ===
SPK051YTNPO786M001:앤드루 김 전 미국 중앙정보국, (CIA)/(씨 아이 에이) 코리아미션센터장은 북한이 미국과 대화 의지가 있고 행동 대 행동 접근법 등 미국의 호의적인 입장 표명을 기다릴지 모른다고 밝혔습니다.
SPK051YTNPO786M003:김 전 센터장은 현지 시각 (5)/(오) 일 미국 워싱턴타임스재단이 주최한 화상 세미나에 참석해 '북한이 미국을 향해 완전한 도발의 사이클 대신 여전히 로키를 보이고 있다' 며 이 같이 말했습니다.
SPK051YTNPO910M003:연도별로 보면, 문재인 정부 출범 이전인 지난 (2016)/(이천 십 육) 년에는 (47)/(마흔 일곱) 건이었지만, 지난해에는 (4)/(네) 배 이상인 (204)/(이백 네) 건의 공표 전 통계가 청와대에 제공됐습니다.
SPK051YTNPO910M002:국민의힘 배준영 의원이 통계청에서 받은 자료를 보면, 지난 (2017)/(이천 십 칠) 년에서 (2021)/(이천 이십 일) 년까지 청와대 비서실이 통계청에서 받은 공표 전 통계는 모두 (813)/(팔백 열 세) 건이었습니다.
SPK051YTNPO708M006:의협 관계자는 '정부 입장에 변화가 없다면 의사들은 코로나 (19)/(일 구) 방역과 진료 현장에서 파업에 내몰릴 것' 이라고 말했습니다.


=== text_preprocessed.txt ()/() 패턴이 포함된 줄중에서 첫 5줄 ===
SPK051YTNPO786M001:앤드루 김 전 미국 중앙정보국, (CIA)/(씨 아이 에이) 코리아미션센터장은 북한이 미국과 대화 의지가 있고 행동 대 행동 접근법 등 미국의 호의적인 입장 표명을 기다릴지 모른다고 밝혔습니다. $앤드루 김 전 미국 중앙정보국, (CIA)/(씨 아이 에이) 코리아미션센터장은 북한이 미국과 대화 의지가 있고 행동 대 행동 접근법 등 미국의 호의적인 입장 표명을 기다릴지 모른다고 밝혔습니다.
SPK051YTNPO786M0